In [1]:
import pandas as pd
import numpy as np
import os
import re
import warnings
import pickle
import os.path
import seaborn
import pickle
import os.path
from numpy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import jieba


mpl.rcParams.update({'font.size': 30})

mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['font.serif'] = ['SimHei']

import seaborn as sns
#sns.set_style('whitegrid',{'font.sans-serif':['simhei','Arial']})
sns.set_style("darkgrid",{"font.sans-serif":['simhei', 'Arial']})
#plt.style.use('ggplot')
mpl.rc('xtick', labelsize=20) 
mpl.rc('ytick', labelsize=20) 
%matplotlib inline

In [2]:
os.chdir("/home/zhh/文档/阿里巴巴/data/6.合表/")
os.listdir()

['总表v2.pickle',
 '金额特征.pickle',
 '金额细节.pickle',
 '出院诊断病种分词将结果.pickle',
 '医院个数.pickle',
 '出院诊断病种的个数.pickle',
 '周索引.pickle',
 '总表v1.pickle',
 '天索引.pickle',
 '交易间隔.pickle',
 '交易次数.pickle']

In [3]:
fee_feature = pd.read_pickle("金额特征.pickle")

In [4]:
fee_detail = pd.read_pickle("金额细节.pickle")

# 修改一下fee_detail的总价和三目统计项目
fee_detail = fee_detail.rename(columns = {"总价":"三目项目总价","三目统计项目":"三目统计项目总计和"})

In [5]:
# 金额特征和金额细节合并成fee表
fee = pd.merge(fee_feature,fee_detail,how='left',left_on=['个人编码','标签'],right_on=['个人编码','标签'])

In [6]:
# 删除金额特征和金额特征
del fee_detail
del fee_feature

In [7]:
cyzd = pd.read_pickle("出院诊断病种分词将结果.pickle")
cyzd.head()
cyzd["标签"] = cyzd["标签"].map(lambda x:1 if x>0 else 0)

In [8]:
cyzd.columns=['个人编码','挂号次数和','空值次数和','肾病次数和','门特次数和','心血管病次数和',
 '脑血管病次数和','癌症次数和','肺病次数和','糖尿病次数和','神经病次数和','精神病次数和','高血压次数和',
 '病变次数和','偏瘫次数和','心脏病次数和','肺心病次数和', '肾性次数和','合并次数和','肾病1次数和','其他次数和', '标签']

In [9]:
# 合并出院诊断病种
fee_cyzd = pd.merge(fee,cyzd,how='left')

In [10]:
# 删除fee和cyzd表
del fee
del cyzd

In [11]:
hs_cont = pd.read_pickle("出院诊断病种的个数.pickle")

In [12]:
#合并出院诊断病种个数
fee_hs_cyzd=pd.merge(fee_cyzd,hs_cont,how ='left')

In [13]:
zhou = pd.read_pickle('周索引.pickle')
tian = pd.read_pickle('天索引.pickle')
jyjg=pd.read_pickle('交易间隔.pickle')
jycs = pd.read_pickle('交易次数.pickle')

In [14]:
data0 = pd.merge(fee_hs_cyzd,zhou,how='left')

data1 =pd.merge(data0,tian[["个人编码","标签","上中下旬"]],how = 'left')

data2 = pd.merge(data1,jyjg,how='left')

data3 = pd.merge(data2,jycs,how='left')

data3.to_pickle("总表v1.pickle")

In [19]:
data3.shape

(15960, 60)

In [21]:
hs_count=pd.read_pickle("医院个数.pickle")

In [24]:
data4 = pd.merge(data3,hs_count,how='left')

In [25]:
data4.to_pickle("总表v2.pickle")

In [3]:
new_week = pd.read_pickle("/home/zhh/文档/阿里巴巴/data/6.合表/新周索引.pickle")
data4 = pd.read_pickle("/home/zhh/文档/阿里巴巴/data/6.合表/总表v2.pickle")

In [4]:
data4.drop(["一周星期几"],axis=1,inplace=True)

In [6]:
data5 = pd.merge(data4,new_week,how='left')

In [7]:
data5.head()
data5.to_pickle("/home/zhh/文档/阿里巴巴/data/6.合表/训练集总表v3.pickle")

In [8]:
data5.head()

,个人编码,药品费发生金额,贵重药品发生金额,中成药费发生金额,中草药费发生金额,药品费自费金额,药品费申报金额,检查费发生金额,检查费申报金额,治疗费发生金额,...,交易间隔最大值,交易次数,医院个数,一周星期几_Friday,一周星期几_Monday,一周星期几_Saturday,一周星期几_Sunday,一周星期几_Thursday,一周星期几_Tuesday,一周星期几_Wednesday
0,352120000000231,9.208056,6.900025,7.980256,-13.815511,3.068983,9.192798,5.192957,5.192957,4.060443,...,2.639058,19,38,4,1,2,0,2,8,2
1,352120000000386,8.842352,7.457194,6.613760,-13.815511,2.916689,8.816529,5.164786,5.164786,4.110874,...,2.708051,9,38,0,1,0,0,0,7,1
2,352120000000408,8.757120,-13.815511,7.207934,-13.815511,-2.525716,8.741202,-13.815511,-13.815511,7.307873,...,2.564950,23,55,0,10,0,0,1,9,3
3,352120000000409,8.231839,-13.815511,8.132516,-13.815511,-13.815511,8.231839,-13.815511,-13.815511,8.284757,...,2.708051,14,35,8,2,1,0,1,2,0
4,352120000000511,9.060940,7.236498,7.731527,-13.815511,-0.328503,9.052959,-13.815511,-13.815511,4.941642,...,2.944440,15,39,3,0,1,1,4,2,4
